## Experiments on _RNAGCN/nRC_ datasets with improved architecture

In [ ]:
import numpy as np
from tensorflow import keras
from sklearn import preprocessing

# Load labels and encodes as categories

train_labels=np.load("encoded_data/dataset_nRC_train_labels.npy")
test_labels=np.load("encoded_data/dataset_nRC_test_labels.npy")

num_classes = len(np.unique(train_labels))
print('Total classes: ',num_classes)

le = preprocessing.LabelEncoder()
le.fit(train_labels)
train_labels_num = le.transform(train_labels)
test_labels_num = le.transform(test_labels)

train_labels_bin = keras.utils.to_categorical(train_labels_num, num_classes)
test_labels_bin = keras.utils.to_categorical(test_labels_num, num_classes)


In [ ]:
from utils.ExpConfiguration import *
from utils.modelUtils import *
import numpy as np
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import f1_score
from numpy.random import seed
seed(1)
from tensorflow.random import set_seed
set_seed(2)

print(tf.__version__)


p ='new' # CHANGE HERE to select other padding schemas (new, constant, random)



batch_size = 32
epochs = 25

# where results are stored
v_acc = {}

for en in seqEncoders:
    print(en['filename'])
    train_seq=np.load('encoded_data/dataset_nRC_train_' + en['filename'] + '_'+p+'_seq.npy')
    test_seq=np.load('encoded_data/dataset_nRC_test_' + en['filename'] + '_'+p+'_seq.npy')

    train_seq = keras.utils.to_categorical(train_seq)
    test_seq = keras.utils.to_categorical(test_seq)

    tf.keras.backend.clear_session()
    print(train_seq.shape)
    print(test_seq.shape)
    if (en['filename'] in ['1mer','2mer','3mer']):
        m=buildCNNModelImproved1D(inshape=train_seq.shape[1:],num_classes=num_classes)
    else:
        m=buildCNNModelImproved2D(inshape=train_seq.shape[1:],num_classes=num_classes)
    m.compile(optimizer=keras.optimizers.Adam(lr=.0005,amsgrad=True),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    m.summary()
    m.fit(train_seq, train_labels_bin,verbose=1,
                  shuffle=True, #batch_size=batch_size,
                  epochs=epochs #,validation_split=0.1
         )

            
    pred = m.predict(test_seq, verbose=1)
    predicted = np.argmax(pred, axis=1)
    v_acc.update({en['filename'] : {'0' : predicted}})



In [ ]:
import pickle

# save results on file
f = open('results/RNAGCN_nRC_ModelImproved_' + p + '.pckl', 'wb')
pickle.dump(v_acc, f)
f.close()


